In [2]:
import pandas as pd
import numpy as np
import requests

In [154]:
CLIENT_ID = 'ND0AF0KVLBEM341PVLKK314M0XKG1CLGTTN42BCT2KJLGQT2' # your Foursquare ID
CLIENT_SECRET = 'FILOMWFFVSVZ2RE4DQQAUXTKT5P2RWQBEGAMYVB4NRPKUON3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [181]:
def getVenues(city, categoryId, radius=1000, limit=50, sortByPopularity=1, query="bar"):

    # create the API request URL
    url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&near={city}&radius={radius}&limit={limit}&             categoryId{section}&sortByPopularity{sortByPopularity}&query={query}"

    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    venues=pd.DataFrame(columns=['popularity_ranking', 'name', 'category'])

    for i, item in enumerate(results):
        venues.loc[i, 'popularity_ranking'] = i+1
        venues.loc[i, 'name'] = item['venue']['name']
        venues.loc[i, 'category'] = item['venue']['categories'][0]['name']
    return venues, results

In [182]:
df, results = getVenues("Amsterdam", "56aa371be4b08b9a8d5734db")

In [134]:
city = "Amsterdam"
radius=2000
limit=50
categoryId="4bf58dd8d48988d116941735" # Id of the food category
sortByPopularity=1

# create the API request URL
url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&near={city}&radius={radius}&             catetgoryId{categoryId}"

# make the GET request
results = requests.get(url).json()['response']['groups'][0]['items']
venues=pd.DataFrame(columns=['popularity_ranking', 'name', 'category'])

for i, item in enumerate(results):
    venues.loc[i, 'popularity_ranking'] = i+1
    venues.loc[i, 'name'] = item['venue']['name']
    venues.loc[i, 'category'] = item['venue']['categories'][0]['name']
venues

,popularity_ranking,name,category
0,1,De Bierkoning,Beer Store
1,2,Kaagman & Kortekaas,French Restaurant
2,3,Proeflokaal De Drie Fleschjes,Bar
3,4,W Amsterdam,Hotel
4,5,Caffè Il Momento,Café
5,6,Tales & Spirits,Cocktail Bar
6,7,Hummingbird,Coffee Shop
7,8,Melly's Espresso - Cookies Bar,Dessert Shop
8,9,Proeflokaal Arendsnest,Beer Bar
9,10,Prik,Gay Bar


In [183]:
df

,popularity_ranking,name,category
0,1,Café Belgique,Bar
1,2,Bierproeflokaal In de Wildeman,Bar
2,3,Bar Jones,Bar
3,4,Café 't Smalle,Bar
4,5,Café Brandon,Bar
5,6,Proeflokaal De Drie Fleschjes,Bar
6,7,Café Chris,Bar
7,8,Hoppe,Bar
8,9,Café 't Spui-tje,Bar
9,10,Tales & Spirits,Cocktail Bar
